# Introduction
This is the implementation of [Age and Gender Classification by Gil Levi and Tal Hassner](https://talhassner.github.io/home/projects/cnn_agegender/CVPR2015_CNN_AgeGenderEstimation.pdf) using PyTorch as the deep learning framework.

The network proposed in the paper has five convolutional layers and three fully connected layers.

## Imports

In [ ]:
import torch
import torch.autograd.variable as Variable
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as utils
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from shutil import copyfile
import pandas as pd
import seaborn as sns

# Preparing dataloaders

## Data

We use the Adience dataset with consists of unfiltered faces.

## Folds

All five folds used in this paper are present [here](https://github.com/GilLevi/AgeGenderDeepLearning/tree/master/Folds/train_val_txt_files_per_fold). We have uploaded them to kaggle in order to use them more easily.


## Data loading

In [ ]:
PATH_TO_FOLDS= "../input/train-val-txt-files-per-fold/train_val_txt_files_per_fold/"
PATH_TO_DATA = "../input/adiencegender/AdienceGender"
PATH_TO_IMAGE_FOLDERS = PATH_TO_DATA + "/aligned"

### Creating a Dataset class

We create a class **`AdienceDataset`** that extends **`Dataset`**. This class helps us in feeding the input data to the network in minibatches.

In [ ]:
class AdienceDataset(Dataset):
    
    def __init__(self, txt_file, root_dir, transform):
        self.txt_file = txt_file
        self.root_dir = root_dir
        self.transform = transform
        self.data = self.read_from_txt_file()
    
    def __len__(self):
        return len(self.data)

    def read_from_txt_file(self):
        data = []
        f = open(self.txt_file)
        for line in f.readlines():
            image_file, label = line.split()
            label = int(label)
            if 'gender' in self.txt_file:
                label += 8
            data.append((image_file, label))
        return data
    
    def __getitem__(self, idx):
        img_name, label = self.data[idx]
        image = Image.open(self.root_dir + '/' + img_name)
        
        if self.transform:
            image = self.transform(image)
            
        return {
            'image': image,
            'label': label
        }         

### Transforms
Every image is first resized to a `256x256` image and then cropped to a `227x227` image before being fed to the network.

**`transforms_list`** is the list of transforms we would like to apply to the input data. Apart from training the neural network without any transformations, we can also train the network using the following transforms (also called as data augmentation techniques):
*   random horizontal flip
*   random crop and random horizontal flip

We don't perform any transformation on the images during validation and testing.


In [ ]:
transforms_list = [
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomCrop(227)
]

transforms_dict = {
    'train': {
        0: list(transforms_list[i] for i in [0, 1, 3]),        # no transformation
        1: list(transforms_list[i] for i in [0, 1, 2, 3]),     # random horizontal flip
        2: list(transforms_list[i] for i in [0, 4, 2, 3])      # random crop and random horizontal flip
    },
    'val': {
        0: list(transforms_list[i] for i in [0, 1, 3])
    },
    'test': {
        0: list(transforms_list[i] for i in [0, 1, 3])
    }
}

### Dataloader
The **`DataLoader`** class in PyTorch helps us iterate through the dataset. This is where we input **`minibatch_size`** to our algorithm.

In [ ]:
def get_dataloader(source, fold, transform_index, minibatch_size):
    """
    Args:
        source: A string. Equals either "train", "val", or "test".
        fold: An integer. Lies in the range [0, 4] as there are five folds present.
        transform_index: An integer. The transforms in the list correesponding
            to this index in the dictionary will be applied on the images.
        minibatch_size: An integer.

    Returns:
        An instance of the DataLoader class.
    """
    path_to_folds= "../input/train-val-txt-files-per-fold/train_val_txt_files_per_fold/"
    path_to_data = "../input/adiencegender/AdienceGender"
    root_dir = path_to_data + "/aligned"
    
    txt_file = f'{path_to_folds}/test_fold_is_{fold}/age_{source}.txt'
    transformed_dataset = AdienceDataset(txt_file, root_dir,
                                         transforms.Compose(transforms_dict[s][transform_index]))
    dataloader = DataLoader(transformed_dataset, batch_size=minibatch_size, shuffle=True, num_workers=4)
    
    return dataloader

# Network

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Defining the network
This is the network as described in the [paper](https://talhassner.github.io/home/projects/cnn_agegender/CVPR2015_CNN_AgeGenderEstimation.pdf).

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 96, 7, stride = 4, padding = 1)
        self.pool1 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm1 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        self.conv2 = nn.Conv2d(96, 256, 5, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm2 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        self.conv3 = nn.Conv2d(256, 384, 3, stride = 1, padding = 1)
        self.pool3 = nn.MaxPool2d(3, stride = 2, padding = 1)
        self.norm3 = nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75)
        
        self.fc1 = nn.Linear(18816, 512)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(512, 512)
        self.dropout2 = nn.Dropout(0.5)
  
        self.fc3 = nn.Linear(512, 10)
    
        self.apply(weights_init)

    
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = self.pool1(x)
        x = self.norm1(x)

        x = F.leaky_relu(self.conv2(x))
        x = self.pool2(x)
        x = self.norm2(x)
      
        x = F.leaky_relu(self.conv3(x))
        x = self.pool3(x)
        x = self.norm3(x)
      
        x = x.view(-1, 18816)
        
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dropout1(x)
      
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.dropout2(x)
      
        x = F.log_softmax(self.fc3(x), dim=1)
  
        return x

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0, std=1e-2)

In [ ]:
criterion = nn.NLLLoss()

## Training the network
We compute and save the train and validation loss after every **`epoch`**. 

We decrease the learning by a tenth after 10,000 iterations using the **`MultiStepLR`** class of PyTorch.

In [ ]:
def train(net, train_dataloader, validation_dataloader, optimizer, scheduler):
    """
    Args:
        net: An instance of PyTorch's Net class.
        train_dataloader: An instance of PyTorch's Dataloader class.
        epochs: An integer.
        validation_dataloader: An instance of PyTorch's Dataloader class.
        optimizer:
        scheduler:
    
    Returns:
        net: An instance of PyTorch's Net class. The trained network.
        training_loss: represents the training loss.
        validation_loss:  represents the validation loss.
    """
    net.train()
    
    checkpoint = 0
    iteration = 0
    running_loss = 0
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        optimizer.step()
        scheduler.step()
        images, labels = batch['image'].to(device), batch['label'].to(device)
        outputs = net(images)
        loss = criterion(outputs, labels)
        running_loss += float(loss.item())
        loss.backward()
        optimizer.step()
    
    training_loss = running_loss / (i+1)
    validation_loss = validate(net, validation_dataloader)
    return net, training_loss, validation_loss

## Validation
We evaluate the performance (in terms of loss) of the trained network on validation set.

In [ ]:
def validate(net, dataloader):
    net.train()
    total_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            total_loss += float(loss.item())

    return total_loss/(i+1)

## Testing
We evaluate the performance (in terms of accuracy) of the trained network on the test set.

In [ ]:
def test(net, dataloader):
    result = {
        'exact_match': 0,
        'total': 0,
        'one_off_match': 0
    }

    with torch.no_grad():
        net.eval()
        for i, batch in enumerate(dataloader):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = net(images)
            outputs = torch.tensor(list(map(lambda x: torch.max(x, 0)[1], outputs))).to(device)
            result['total'] += len(outputs)
            result['exact_match'] += sum(outputs == labels).item()
            result['one_off_match'] += (sum(outputs==labels) + sum(outputs==labels-1) + sum(outputs==labels+1)).item()

    return result['total'], result['exact_match'], result['one_off_match']

# Execution

## Hyperparameters
The **`minibatch_size`** and **`lr`** are pulled from the paper, **`num_epochs`** is set empirically. 

In [ ]:
lr = 0.0001  # initial learning rate
epochs = 40
minibatch_size = 50

## Running the model with different folds and transformations

##### We run each of the folds 0, 1, 2, 3, 4 - each with each of the transormations options for the train

In [ ]:
folds = [0, 1, 2, 3, 4]
train_transform_indexs = [0, 1, 2]
test_results_by_fold_transform_epoch = {}

for fold in folds:
    for train_transform_index in train_transform_indexs:
        print("=" * 30)
        print("Running the model with: fold: " + str(fold) + ", train transform: " + str(train_transform_index))
        
        # setup
        criterion = nn.NLLLoss().to(device)
        new_model= Net().to(device)
        lr = 0.0001  # initial learning rate
        optimizer = optim.Adam(new_model.parameters(), lr)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10000])

        test_results_by_fold_transform_epoch[(fold, train_transform_index)] = {'exact_match': [],
                                                                               'total': [],
                                                                               'one_off_match': []}
        training_loss = []
        validation_loss = []
        train_loader = get_dataloader('train', fold, train_transform_index, minibatch_size)
        validation_loader = get_dataloader('val', fold, 0, minibatch_size)
        test_loader = get_dataloader('test', fold, 0, minibatch_size)
        
        # runing a train and a test for each epoch, collecting and printing the results
        for epoch in range(epochs):
                new_model, curr_training_loss, curr_validation_loss = train(new_model, train_loader, validation_loader, optimizer, scheduler)
                training_loss.append(curr_training_loss)
                validation_loss.append(curr_validation_loss)
                total, exact_match, one_off_match = test(new_model, test_loader)
                test_results_by_fold_transform_epoch[(fold, train_transform_index)]['total'].append(total)
                test_results_by_fold_transform_epoch[(fold, train_transform_index)]['exact_match'].append(exact_match)
                test_results_by_fold_transform_epoch[(fold, train_transform_index)]['one_off_match'].append(one_off_match)
                print("EPOCH: " + str(epoch) + " | Train and Validate: training_error: " + str(training_loss[-1]) + ", validation_error:" + str(validation_loss[-1]) + 
                      " | Test: total: " + str(total) + ", exact match: " + str(exact_match) + ", one off match: " + str(one_off_match))
        
        # plotting the results
        plt.figure(figsize=(12,6))

        plt.xticks(range(epochs))
        plt.plot(training_loss, label='training_loss')
        plt.plot(validation_loss, label='validation_loss')
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=2, mode="expand", borderaxespad=0.)
        plt.xlabel('Epochs')
        plt.ylabel('loss')
        plt.show()

        test_results = test_results_by_fold_transform_epoch[(fold, train_transform_index)]

        result_dataframe = pd.DataFrame(
            {'total': test_results['total'],
             'one_off_match': test_results['one_off_match'],
             'exact_match': test_results['exact_match']})
        sns.set(rc={'figure.figsize':(12,6)})
        sns.lineplot(data=result_dataframe)

# Results